# 使用教學  

> <font size=4> 1. 請先和系辦曾小姐索取今年申請本系轉系、輔系或雙主修的學生成績檔案</font>   
> <font size=4> 2. 若成績檔的檔案類型為.xls檔，請將其另存為.xlsx檔</font>    
> <font size=4> 3. 按下左上方的「」按鍵將這些新的xlsx檔上傳</font>    
> <font size=4> 4. 到"設定變數"下的程式碼當中尋找變數grade_path，分別設定申請轉系、輔系或雙主修學生的成績檔的名稱</font>    
> <font size=4> 5. 按下上方的「」按鍵執行以下3組程式碼</font>    
> <font size=4> 6. 等程式跑完以後，在左方的檔案區中找到名稱含有"results"的xlsx檔案後，點右鍵下載至電腦</font>    
> <font size=4> 7. 嘗試手動計算2~3位學生的平均成績並與結果檔案中記錄的平均成績進行比對</font>    
> <font size=4> 8. 若無問題即可將結果寄給系辦曾小姐</font>  
>     
> <font size=3> 若有任何問題請聯繫作者施彥任(F08524004@ntu.edu.tw)</font>    
> <font size=3> 最後編輯時間：2022/7/11</font>  

# 載入模組

In [2]:
from ntuche_tmdm import arrangement

# 設定變數

In [2]:
"""
Parameters
----------
core_course1 : list, str
    本校所有微積分、普通化學與普通物理學課名的共通字串
grade_path: list, str
    存放申請輔系、轉系與雙主修學生的成績檔案儲存路徑
"""
core_course1= ['微積分', '普通化學', '普通物理學']
grade_path = ['111輔系.xlsx', '111轉系.xlsx', '111雙主修.xlsx']

# 執行計算並儲存為excel檔

In [3]:
"""
Parameters
----------
    存放申請輔系、轉系與雙主修學生的成績檔案儲存路徑
savepathi: str
    結果檔案的儲存路徑
studentrank: object of class "arrangement"
    執行所有計算的物件
sheetnamei: str
    結果檔案的工作表名稱
"""

for grade_pathi in grade_path:
    grade_path1, grade_path2 = grade_pathi.split('.')
    savepathi = grade_path1 + '_results.' + grade_path2
    studentranki = arrangement(grade_pathi, core_course1)
    sheetnamei = 'results'
    studentranki.save_df_data(studentranki.df_alldata, savepathi, sheetnamei)

### 專門跑本系學生成績的程式碼(舊程式碼，可忽略)

In [23]:
class arrangement:
    """
    Parameters
    ----------
    all_paths : tuple, str
        大二、大三和大四學生成績路徑 
    core_course2 : list, str
        化工系所有必修課的核心課程科目(大二與大三同分參酌二的科目)
    che_course_number: list, str
        化工系開設課程之課號前三碼
    """
    
    course_calculus = '微積分'
    course_chemistry = ['普通化學甲','普通化學乙','普通化學丙','普通化學丁']
    course_physics = ['普通物理學甲','普通物理學乙']
    
    def __init__(self, grade_path):
        """
        
        初始化
        
        ----------
        Parameters
        ----------
        studentgrade: int
            學生年級
        path : str
            學生成績的檔案路徑
        __df_alldata : pd.DataFrame
            所有學生的所有平均分數資料
        """
        self.grade_path = grade_path
        self.__df_alldata = None
    
    @staticmethod
    def dedupe(items):
        """
        
        在不影響順序的情況下刪除重複元素的函數
        
        ----------
        Parameters
        ----------
        items: iterable list, int
            有許多重複元素的list或array
        """
        seen = set()
        for item in items:
            if item not in seen:
                yield item
                seen.add(item)
    
    @property
    def df_gradedata(self): #學生成績
        """
        Parameters
        ----------
        df_gradedata: pd.DataFrame
            所有學生的成績總表
        col: list, str
            成績總表的欄位名稱
        """
        df_gradedata = pd.read_excel(self.grade_path).replace('\xa0\xa0', np.nan)
        sheetname1 = df_gradedata.columns[0]
        col = [str(i1)+str(i2) if type(i2) is str else i1 for i1, i2 in zip(df_gradedata.iloc[1], df_gradedata.iloc[2])] #整理獲得column名稱
        df_gradedata.columns = col
        df_gradedata = df_gradedata.iloc[3:].reset_index(drop=True) #刪除dataframe內的column名稱
        return df_gradedata, sheetname1
    
    @property
    def df_gradedata_split(self): #學生成績
        """
        Parameters
        ----------
        df_gradedata: pd.DataFrame
            所有學生的成績總表
        col: list, str
            成績總表的欄位名稱
        """
        df_gradedata, sheetname1 = self.df_gradedata
        df_gradedata_split = []
        sheetname = []
        sheetname.append(sheetname1)
        split_linenum = []
        for i, line_data in enumerate(df_gradedata['學年學期']):
            if type(line_data) is str:
                if ( line_data.strip()[-3:] in ['上學期', '下學期'] ) or ( line_data.strip()[-2:] == '暑修' ) or ( line_data.strip()[-4:] == '免修抵免' ):
                    split_linenum.append(i)
                    sheetname.append(line_data)
        df_gradedata_split.append(df_gradedata.iloc[:split_linenum[0]-3])
        for i, n in enumerate(split_linenum[:-1]):
            df_gradedata_split.append(df_gradedata.iloc[split_linenum[i]+4:split_linenum[i+1]-3].reset_index(drop=True))
        del sheetname[-1]
        return df_gradedata_split, sheetname
    
    @property
    def all_students_id_split(self): 
        """
        Parameters
        ----------
        all_students_id: list, str
            所有學生的學號
        """
        df_gradedata_split, sheetname = self.df_gradedata_split
        all_students_id_split = []
        for df_gradedata_spliti in df_gradedata_split:
            all_students_id_split.append([i.strip() for i in self.dedupe((df_gradedata_spliti['學號']))])
        return all_students_id_split
    
    @property
    def all_students_id(self): 
        """
        Parameters
        ----------
        all_students_id: list, str
            所有學生的學號
        """
        all_students_id_split = self.all_students_id_split
        all_students_id = list(self.dedupe(sum(all_students_id_split, [])))
        return all_students_id
    
    @property
    def all_students_name_split(self):
        """
        Parameters
        ----------
        all_students_name: list, stra
            所有學生的姓名
        """
        df_gradedata_split, sheetname = self.df_gradedata_split
        all_students_name_split = []
        for df_gradedata_spliti in df_gradedata_split:
            all_students_name_split.append([i.strip() for i in self.dedupe((df_gradedata_spliti['姓名']))])
        return all_students_name_split
    
    @property
    def all_students_name(self):
        """
        Parameters
        ----------
        all_students_name: list, stra
            所有學生的姓名
        """
        all_students_name_split = self.all_students_name_split
        all_students_name = list(self.dedupe(sum(all_students_name_split, [])))
        return all_students_name
    
    @property
    def all_students_department_split(self):
        """
        Parameters
        ----------
        all_students_name: list, stra
            所有學生的姓名
        """
        df_gradedata_split, sheetname = self.df_gradedata_split
        all_students_id_split = self.all_students_id_split
        all_students_department_split = []
        for df_gradedata_spliti, all_students_idi in zip(df_gradedata_split, all_students_id_split):
            all_students_department = []
            for students_id in all_students_idi:
                try:
                    students_department = df_gradedata_spliti[df_gradedata_spliti['學號']==students_id+'\xa0']['系所組名稱'].iloc[0].strip()
                except:
                    students_department = np.nan
                all_students_department.append(students_department)
            all_students_department_split.append(all_students_department)
        return all_students_department_split
    
    @property
    def all_students_department(self):
        """
        Parameters
        ----------
        all_students_name: list, stra
            所有學生的姓名
        """
        df_gradedata_split, sheetname = self.df_gradedata_split
        all_students_id = self.all_students_id
        all_students_department = []
        for students_id in all_students_id:
            students_department = None
            for df_gradedata_spliti in df_gradedata_split:
                try:
                    students_department = df_gradedata_spliti[df_gradedata_spliti['學號']==students_id+'\xa0']['系所組名稱'].iloc[0].strip()
                except:
                    students_department = None
                if students_department is not None:
                    all_students_department.append(students_department)
                    break
        return all_students_department
    
    def calc_allavg(self, student_id, full_output=False):
        """
        
        計算一個學生所有科目的總平均(大二與大三的同分參酌一、大四的排名標準)
        
        ----------
        Parameters
        ----------
        student_id: str
            學生的學號
        full_output: boolean
            是否需要輸出學生的總學分數
        df_gradedata: pd.DataFrame
            所有學生的成績總表
        grade: np.array, float
            學生各科的成績(等第積分)
        credit: np.array, int
            學生各科的學分數
        flag: boolean
            是否已搜尋到此學生的資料
        allcredit: int
            學生的總學分數
        allavg: float
            學生所有科目的平均分數
        """
        df_gradedata_split, _ = self.df_gradedata_split
        allcredits = np.array([])
        allavgs = np.array([])
        for df_gradedata_spliti in df_gradedata_split:
            grade = np.array([])
            credit = np.array([])
            flag = False
            for (ide, cde, gde) in zip(df_gradedata_spliti['學號'], df_gradedata_spliti['學分'], df_gradedata_spliti['等第績分']):
                if ide.strip() == student_id and not np.isnan(float(gde)):
                    grade = np.append(grade, float(gde.strip()))
                    credit = np.append(credit, int(cde.strip()))
                    flag = True
                if ide.strip() != student_id and flag:
                    break
            allcredit = sum(credit)
            allavg = np.sum( grade * credit ) / allcredit if ( grade.size != 0 ) and ( allcredit != 0 ) else 0
            allavg = self.modify_round(allavg)
            allcredits = np.append(allcredits, allcredit)
            allavgs = np.append(allavgs, allavg)
        if full_output:
            return allavgs, allcredits
        else:
            return allavgs
    
    def calc_core1avg(self, student_id, full_output=False):
        """
        
        計算一個學生必修科目總平均(大二與大三的排名標準)
        
        ----------
        Parameters
        ----------
        student_id: str
            學生的學號
        full_output: boolean
            是否需要輸出學生的各個必修課目的課程名稱、等第成績、等第積分與學分數資料
        df_gradedata: pd.DataFrame
            所有學生的成績總表
        core_course1: list, int
            化工系所有必修課目名稱
        core_course1_name: list, str
            學生修習的各個必修課目名稱
        grade: np.array, float
            學生修習的各個必修課目成績(等第積分)
        credit: np.array, int
            學生修習的各個必修課目的學分數
        gdcddata: list, int
            學生修習的各個必修課目的"等第成績 等第積分 學分數"
        flag: boolean
            是否已搜尋到此學生的資料
        core1credit: int
            學生必修課的總學分數
        core1avg: float
            學生必修課的平均分數
        fulldata: dict, tuple, str
            包含學生修習的各個必修課目的課程名稱、等第成績、等第積分與學分數, 等第成績代表A+, A, A-, ...等
        """
        df_gradedata_split, sheetname = self.df_gradedata_split
        course_calculus = self.course_calculus
        course_chemistry = self.course_chemistry
        course_physics = self.course_physics
        core_course1_name = []
        grade = np.array([])
        credit = np.array([])
        gdcddata = []
        for df_gradedata_spliti, sheetnamei in zip(df_gradedata_split, sheetname):
            flag = False
            for (ide, cne, cde, gne, gde) in zip(df_gradedata_spliti['學號'], df_gradedata_spliti['課程名稱'], df_gradedata_spliti['學分'], df_gradedata_spliti['等第成績'], df_gradedata_spliti['等第績分']):
                if ( ide.strip() == student_id ) and ( not np.isnan(float(gde)) ) and ( ( cne.strip()[:3] in course_calculus ) or ( cne.strip()[:5] in course_chemistry ) or ( cne.strip()[:6] in course_physics ) ) :
                    core_course1_name.append(cne.strip())
                    grade = np.append(grade, float(str(gde).strip()))
                    credit = np.append(credit, int(cde.strip()))
                    gdcddata.append(gne.strip() + ' ' + str(gde).strip() + ' ' + cde.strip())
                    flag = True
                if ide.strip() != student_id and flag:
                    break
        core1credit = sum(credit)
        core1avg = np.sum( grade * credit ) / core1credit if ( grade.size != 0 ) and ( core1credit != 0 ) else 0
        core1avg = self.modify_round(core1avg)
        if full_output:
            fulldata = dict(zip(core_course1_name, gdcddata))
            return core1avg, fulldata
        else:
            return core1avg
    
    
    def get_df_alldata(self):
        """
        
        計算所有學生的所有平均分數資料
        
        ----------
        Parameters
        ----------
        studentgrade: int
            學生年級
        all_students_id: list, str
            所有學生的學號
        all_students_name: list, str
            所有學生的姓名
        all_allavg: list, float
            所有學生的全科目平均分數
        all_allcredit: list, int
            所有學生的總學分數
        all_core1avg: list, float
            所有學生的必修課平均分數
        all_core2avg: list, float
            所有學生的必修課平均分數
        all_core3avg: list, float
            所有學生的化工系課程平均分數
        all_core3_number: list, float
            所有學生修習的化工系課程數目
        df_corse1data: pd.DataFrame
            大二與大三：所有學生修習各個必修課的等第成績、等第積分與學分數總表
            大四：所有學生修習各個化工系課程的等第成績、等第積分與學分數總表
        df_avgdata: pd.DataFrame
            大二與大三：所有學生的必修課平均分數、全科目平均分數(同分參酌一)、必修課的核心課程平均分數(同分參酌二)與總學分數資料表
            大四：所有學生的全科目平均分數、化工系課程平均分數(同分參酌一)、總學分數(同分參酌二)與修習的化工系課程數目資料表
        column: list, str
            df_avgdata的欄位名稱
        df_alldata: pd.DataFrame
            df_avgdata和df_corse1data合併後的總表
        """
        all_students_id = self.all_students_id
        all_students_name = self.all_students_name
        all_students_department = self.all_students_department
        df_gradedata_split, sheetname = self.df_gradedata_split
        all_allavg = []
        all_allcredit = []
        all_core1avg = []
        df_corse1data = pd.DataFrame()
        sheetname_new = []
        for sheetnamei in sheetname:
            sheetname_new.append(sheetnamei+' 所有科目平均')
            sheetname_new.append(sheetnamei+' 總學分數')
        column = ['學號','姓名','系所組名稱'] + sheetname_new + ['三科平均']
        for student_id in all_students_id:
            allavg, allcredit = self.calc_allavg(student_id, True)
            core1avg, fulldata = self.calc_core1avg(student_id, True)
            all_allavg.append(allavg)
            all_allcredit.append(allcredit)
            df_corse1data = pd.concat([df_corse1data, pd.DataFrame([fulldata])], ignore_index=True)
            all_core1avg.append(core1avg)
        data_allavg_allcredit = []
        all_allavg = np.array(all_allavg)
        all_allcredit = np.array(all_allcredit)
        for all_allavgi, all_allcrediti in zip(all_allavg.T, all_allcredit.T):
            data_allavg_allcredit.append(all_allavgi.tolist())
            data_allavg_allcredit.append(all_allcrediti.tolist())
        data = [all_students_id, all_students_name, all_students_department] + data_allavg_allcredit + [all_core1avg]
        df_avgdata = pd.DataFrame(zip(*data), columns=column)
        df_alldata = pd.concat([df_avgdata, df_corse1data], axis=1)
        self.__df_alldata = df_alldata
        return df_alldata
    
    @property
    def df_alldata(self):
        """
        Parameters
        ----------
        df_alldata: pd.DataFrame
            所有學生的所有平均分數資料總表
        """
        if self.__df_alldata is None:
            return self.get_df_alldata()
        else:
            return self.__df_alldata
    
    @property
    def df_rankdata(self):
        """
        
        進行排名的計算
        
        ----------
        Parameters
        ----------
        studentgrade: int
            學生年級
        col_all: list, str
            排名所依照的各種先後順序，順序由左到右
        df_alldata: pd.DataFrame
            所有學生的所有平均分數資料總表
        df_rankdata: pd.DataFrame
            包含所有學生所有平均分數資料的排名總表
        """
        col_all = ['三科平均']
        df_alldata = self.df_alldata
        df_rankdata = df_alldata.copy()
        ranklist = df_rankdata[col_all].apply(tuple, axis=1).rank(method='min', ascending=0)
        df_rankdata.insert(0, '排名', ranklist) #插入一欄紀錄每位學生的排名
        df_rankdata.sort_values('排名', inplace=True) #將此表格以排名來排序
        df_rankdata.index = df_alldata.index
        return df_rankdata
    
    def save_df_data(self, df_data, savepath, sheet_name, method='dataframe_to_rows'):
        """
        
        將排名後的資料儲存至指定路徑
        
        ----------
        Parameters
        ----------
        savepath: str
            排名結果的檔案儲存路徑
        sheet_name: str
            設定結果的excel檔中的工作表名稱
        method: str
            存檔的方法有兩種:
            1. ExcelWriter: 程式碼比較簡潔，但我當初在編寫時有時候會產生出損毀過的excel檔
            2. dataframe_to_rows: 程式碼看起來比較繁雜，但是可以產生出正常的excel檔
        df_rankdata: pd.DataFrame
            包含所有學生所有平均分數資料的排名總表
        """
        if method == 'ExcelWriter':
            if os.path.exists(savepath):
                writer = pd.ExcelWriter(savepath, engine='openpyxl', mode='a')
                book = openpyxl.load_workbook(savepath)
                writer.book = book
            else:
                writer = pd.ExcelWriter(savepath, engine='openpyxl')
                book = openpyxl.Workbook()
            df_data.to_excel(writer, sheet_name=sheet_name, index=False)
            writer.save()
            writer.close()
        elif method == 'dataframe_to_rows':
            rows = dataframe_to_rows(df_data, index=False)
            if os.path.exists(savepath):
                book = openpyxl.load_workbook(savepath)
                sheet = book.create_sheet(title=sheet_name)
            else:
                book = openpyxl.Workbook()
                sheet = book.active
                sheet.title = sheet_name
            for r_idx, row in enumerate(rows, 1):
                for c_idx, value in enumerate(row, 1):
                     sheet.cell(row=r_idx, column=c_idx, value=value)
            book.save(filename=savepath)
        else:
            print('Please input "dataframe_to_rows" or "ExcelWriter" to method variable.')